<a href="https://colab.research.google.com/github/adas754/generative-AI_class/blob/main/mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install llama-index-llms-huggingface
%pip install llama-index

In [2]:
!pip install weaviate-client

In [3]:
WEAVIATE_CLUSTER="https://myliveclass-xpagdq8f.weaviate.network"
WEAVIATE_API_KEY="B2NrLZmFTbNwOJoLKWxxglgfTqDqr9ywkpLq"

In [4]:
import weaviate
client=weaviate.Client(url=WEAVIATE_CLUSTER,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [5]:
client=weaviate.Client(url=WEAVIATE_CLUSTER,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [7]:
!mkdir Data

mkdir: cannot create directory ‘Data’: File exists


In [8]:
loader=SimpleDirectoryReader('/content/Data')

In [9]:
documents=loader.load_data()

In [10]:
documents

[Document(id_='52d5c21b-f789-4a0a-9a2c-66e00b371d5e', embedding=None, metadata={'page_label': '1', 'file_name': 'MachineTranslationwithAttention.pdf', 'file_path': '/content/Data/MachineTranslationwithAttention.pdf', 'file_type': 'application/pdf', 'file_size': 482134, 'creation_date': '2024-04-13', 'last_modified_date': '2024-04-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/355917108\nNeu ral Machine T ranslation with Attention\nTechnic al R eport  · August 2021\nDOI: 10.13140/RG.2.2.29381.37607/1\nCITATIONS\n0READS\n1,579\n2 author s:\nMohammad W asil Saleem\nUniv ersität P otsdam\n3 PUBLICA TIONS \xa0\xa0\xa00 CITATIONS \xa0\xa

In [11]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [12]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model=HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
from llama_index.core import PromptTemplate

In [14]:
from llama_index.llms.huggingface import HuggingFaceLLM

In [15]:
system_prompt=""""<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided

"""

In [16]:
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [17]:
"""
1. System & Assistant (LLM)
2. User
"""

'\n1. System & Assistant (LLM)\n2. User\n'

In [18]:
import torch
llm=HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}

)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
from llama_index.core import ServiceContext

Service_Context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-19-f0b118d025f9>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  Service_Context=ServiceContext.from_defaults(


In [20]:
from llama_index.core import VectorStoreIndex

In [21]:
index=VectorStoreIndex.from_documents(documents,service_context=Service_Context)

In [22]:
query_engine=index.as_query_engine(k=1)

In [23]:
result=query_engine.query("which matrics we can use for evaluating the machine translation task?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [24]:
result.response

'We can use BLEU (Bilingual Evaluation Understudy) score for evaluating the machine translation task. BLEU score is a widely used metric for evaluating the quality of machine translation. It measures the similarity between the generated translation and a set of human reference translations. The higher the BLEU score, the better the quality of the translation. In addition to BLEU score, we can also use other metrics such as perplexity, ROUGE (Recall-Oriented Understudy for Gisting Evaluation), and METEOR (Metric for Evaluation of Translation with Explicit ORdering).'

In [25]:
from IPython.display import Markdown, display

In [26]:
display(Markdown(f"<b>{result}</b>"))

<b>We can use BLEU (Bilingual Evaluation Understudy) score for evaluating the machine translation task. BLEU score is a widely used metric for evaluating the quality of machine translation. It measures the similarity between the generated translation and a set of human reference translations. The higher the BLEU score, the better the quality of the translation. In addition to BLEU score, we can also use other metrics such as perplexity, ROUGE (Recall-Oriented Understudy for Gisting Evaluation), and METEOR (Metric for Evaluation of Translation with Explicit ORdering).</b>

In [27]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [28]:

import weaviate
client=weaviate.Client(url=WEAVIATE_CLUSTER,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [29]:
!pip install llama-index-vector-stores-weaviate

In [30]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore

In [31]:
from llama_index.core import StorageContext

In [32]:
vector_store=WeaviateVectorStore(weaviate_client=client,index_name="Llamaindex")

In [33]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [34]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,embed_model=embed_model
)

In [35]:
query_engine = index.as_query_engine(llm=llm)

In [36]:
result3=query_engine.query("what is attention?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [37]:
result3.response

'Attention is a mechanism used in machine translation models to retain and utilize all the hidden state of the input sentence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence, which helps to improve the translation of the source sentence.'